## Imports

In [ ]:
import azureml.core
# display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

from azureml.core import Workspace
from azureml.core.model import Model
import os 
ws = Workspace.from_config(path='azure_config_dev.json')
print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')



In [ ]:
ml_service = 'PAD'

#TODO - replace with call to HRXML Config API to get algorithm details for web service
if ml_service == 'TWV':
    algorithm = 'DNN_MLPRegressor'
    description = 'Tax Withholding Verification'
elif ml_service == 'PAD':
    algorithm = 'Isolation_Forest'
    description = 'Payroll Anomaly detection'
else:
    algorithm = 'Other algorithm'
    description = 'Other'

###  Choose published models to deploy in web service, and try  to download to ensure they exist

In [ ]:
#models_list =  [{'ml_service':'TWV','model_code':'SS01','model_version':'010','vers':1},{'ml_service':'TWV','model_code':'T001','model_version':'002','vers':1},{'ml_service':'PAD','model_code':'M010','model_version':'002','vers':1},{'ml_service':'PAD','model_code':'M005','model_version':'001','vers':1},{'ml_service':'PAD','model_code':'A001','model_version':'001','vers':5}] #,{'model_code':'T003','model_vers':2}]
models_list =  [{'ml_service':'TWV','model_code':'T001','model_version':'002','vers':1},{'ml_service':'PAD','model_code':'M005','model_version':'001','vers':1}] #,{'model_code':'T003','model_vers':2}]
#ml_service = 'TWV'


twv_models = []

for model_details in models_list:
    ml_serv = model_details['ml_service']
    model = model_details['model_code']  #'T003'
    model_version = None
    if 'model_version' in model_details:
        model_version = model_details['model_version']
    version = model_details['vers']
    if model_version is None:
        twv_model=Model(ws, ml_serv + '_model_' + model,version=version)
    else:
        twv_model=Model(ws, ml_serv + '_model_' + model + '_' + model_version,version=version)
    twv_model.add_tags({'azure_model_vers':version})
    twv_models.append(twv_model)

    print('model: ' + str(twv_model.name) + ' ' + str(twv_model.version))

    ser = twv_model.serialize()
    print('ser: ' + str(ser))
    twv_model.download(target_dir=os.path.join(os.getcwd(),'downloaded_models'), exist_ok=True)

    # verify the downloaded model file
    #file_path = os.path.join(os.getcwd(), "sklearn_mnist_model.pkl")

    #os.stat(file_path)

## Package dependencies

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")
myenv.add_conda_package("pandas")
myenv.add_conda_package("matplotlib")
myenv.add_conda_package("numpy")
myenv.add_conda_package("requests")
myenv.add_channel("conda-forge")
myenv.add_pip_package("azureml-pipeline-core")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())


In [ ]:
with open("myenv.yml","r") as f:
    print(f.read())

## Web service depoyment configuration

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"ml_service": ml_service,"algorithm":algorithm}, 
                                               description=description)

# Option 1 - Use when updating an  existing web service with a new image

In [ ]:
ngamlfpy_package = 'ngamlfpy'

In [ ]:
ml_service = 'TWV' # for image purposes - has both pad and twv modesl in twvimage

In [ ]:
%%time
#rbm code to create new image

from azureml.core.image import Image
from azureml.core.image import ContainerImage

new_img_name = 'mlfpyimage' #ml_service.lower() + 'image'
image_config = ContainerImage.image_configuration(execution_script="predict_model.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml",
                                                 dependencies=[os.path.join('.',ngamlfpy_package)
                                                               #os.path.join('.', 'gen_utils.py'),
                                                               #os.path.join('.', 'pipeline_utils.py'),
                                                               #os.path.join('.',  'train_utils.py')
                                                               ])


img = Image.create(workspace=ws, name=new_img_name, models=twv_models, image_config=image_config)

In [ ]:
#rbm code to get webservice for existing image
from azureml.core.webservice import Webservice

#prev_image_name = 'twvdnn9' 
web_services = Webservice.list(workspace=ws, image_name=None, image_id=None, model_name=None, model_id=None, tags=None, properties=None)
web_services


In [ ]:
web_services[0].name

In [ ]:
images=Image.list(workspace=ws)
images

In [ ]:
new_img = images[0]
new_img

In [ ]:
#rbm code to update existing webservie with new image
web_services[0].update(image=new_img, tags=None, properties=None, description=ml_service + " Prediction", auth_enabled=None, ssl_enabled=None, ssl_cert_pem_file=None, ssl_key_pem_file=None, ssl_cname=None, enable_app_insights=None)

In [ ]:
web_services[0].get_logs()

# ... or Option 2) Use when deplying a new web service (also deploys new image at same time)

In [ ]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

ngamlfpy_package = 'ngamlfpy'

# configure the image
image_config = ContainerImage.image_configuration(execution_script="predict_model.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml",
                                                 dependencies=[
                                                               os.path.join('.',ngamlfpy_package)
                                                               #os.path.join('.', 'gen_utils.py'),
                                                               #os.path.join('.', 'pipeline_utils.py'),
                                                               #os.path.join('.',  'train_utils.py')
                                                               ])

service = Webservice.deploy_from_model(workspace=ws,
                                       #name= ml_service.lower() + 'image',
                                       name= 'mlfpyimage',
                                       deployment_config=aciconfig,
                                       models=twv_models,
                                       image_config=image_config)


service.wait_for_deployment(show_output=True)

## Note: can debug prediction web service with service.get_logs()

In [ ]:
print(service.get_logs())

## Try running web service

In [ ]:
web_services[0].scoring_uri

In [ ]:
import json
model = 'T001'
predict_file_name =  model + '_test.json'
data_dir='data/predict'
predict_source = 'json'
model_prefix = model
predict_file_path = '/'.join([data_dir,model_prefix])
full_predict_file_name = '/'.join([predict_file_path,predict_file_name])
print ('Predict file: ',full_predict_file_name)
if predict_source == 'json':
    with open(full_predict_file_name) as json_data:
        j_data = json.load(json_data)
raw_data = {}
raw_data['data'] = j_data
raw_data['data']

In [ ]:
raw_data_str = json.dumps(raw_data)
test_samples = bytes(raw_data_str, encoding='utf8')
result = web_services[0].run(input_data=test_samples)

result

In [ ]:
# import json


# predict_file_name = model + '_test.json'
# data_dir='data/predict'
# predict_source = 'json'
# model_prefix = model + '_' + target_vers
# predict_file_path = '/'.join([data_dir,model_prefix])
# full_predict_file_name = '/'.join([predict_file_path,predict_file_name])
# print ('Predict file: ',full_predict_file_name)
# if predict_source == 'json':
#     with open(full_predict_file_name) as json_data:
#         j_data = json.load(json_data)
        

# test_samples = json.dumps({"data": j_data})
# test_samples = bytes(test_samples, encoding='utf8')

# # predict using the deployed model
# result = service.run(input_data=test_samples)
# result